In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import sys
sys.path.append('../src/')
from plotutils import plotts
from wahooreader import WahooTickrX
from bokeh.io import output_notebook
from datetime import datetime
import glob
import os
import pandas as pd
output_notebook()

Loading BokehJS ...

# Heart Rate Recovery Trends

In [13]:
fit_files = glob.glob('/Users/hasannagib/Dropbox/Apps/WahooFitness/*.fit')

In [14]:
dates = []
recs = []
for fit in fit_files:
    dates.append(pd.to_datetime(os.path.basename(fit)[:17]))
    wtx = WahooTickrX(fit)
    recs.append(dict(wtx.heartrate.max()[['60_sec_rec', '120_sec_rec', '180_sec_rec']]))

Looks like `2020-06-20` had recovery heart rate anomaly. It will be excluded. The 4 recovery targets are taken from [this](https://support.fitdigits.com/tips-and-how-tos/how-fit-are-you-take-the-2-min-recovery-test/) article.

In [72]:
df = pd.DataFrame(recs)
df = df[['120_sec_rec']]
df['timestamp'] = dates
df['L0'] = 52
df['L1'] = 58
df['L2'] = 65
df['L3'] = 65
df = df.set_index('timestamp').sort_index().query('timestamp != ("2020-06-20 11:43:01")')
plotts(df, styles=['o-']+4*['-'], plot_width=600, ymin=20);

# Inspect `.fit` file

In [67]:
# Get .fit file for a given date
date = '2020-09-09'
file = glob.glob(f'/Users/hasannagib/Dropbox/Apps/WahooFitness/*{date}*.fit')
file

['/Users/hasannagib/Dropbox/Apps/WahooFitness/2020-09-09-110155-FITNESS FB34-28-0.fit']

In [68]:
# Parse fit file
wtx = WahooTickrX(file[0])

In [69]:
plotts(wtx.heartrate, plot_width=600);

In [71]:
wtx.heartrate.max()

heart_rate     187.0
60_sec_rec      42.0
120_sec_rec     65.0
180_sec_rec     64.0
dtype: float64